In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.optimizers import Adam, RMSprop, Adagrad, Adamax, Nadam, Ftrl
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
from tensorflow.keras.models import Model
from catboost import CatBoostRegressor

class CatBoostLayer(Model):
    def __init__(self, **kwargs):
        super(CatBoostLayer, self).__init__(**kwargs)
        self.catboost = CatBoostRegressor(silent=True)

    def fit(self, X, y):
        self.catboost.fit(X, y)

    def predict(self, X):
        return self.catboost.predict(X)

    def call(self, inputs):
        return self.predict(inputs)



In [ ]:
from keras.models import Model, Input
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import l2

def EcoCurrentNet_V1(input_shape):
    model_input = Input(shape=(input_shape,))
    
    # 第一层
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(model_input)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # 第二层
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # 第三层
    x = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # 输出层
    output = Dense(1)(x)

    model = Model(inputs=model_input, outputs=output)
    catboost = CatBoostLayer()
    
    return model, catboost


In [ ]:
from keras.models import Model, Input
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.layers import LeakyReLU

def EcoCurrentNet_V2(input_shape):
    model_input = Input(shape=(input_shape,))
    
    # 第一层
    x = Dense(256, kernel_regularizer=l2(0.01))(model_input)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dropout(0.3)(x)

    # 第二层
    x = Dense(128, kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dropout(0.3)(x)

    # 第三层
    x = Dense(64, kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    # 第四层
    x = Dense(32, kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    # 输出层
    output = Dense(1)(x)

    model = Model(inputs=model_input, outputs=output)
    catboost = CatBoostLayer()
    return model, catboost


In [ ]:
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2

def EcoCurrentNet_V3(input_shape):
    model_input = Input(shape=(input_shape, 1))
    
    # 第一层卷积
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(model_input)
    x = MaxPooling1D(pool_size=2)(x)
    x = BatchNormalization()(x)
    
    # 第二层卷积
    x = Conv1D(filters=128, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = BatchNormalization()(x)
    
    # Flatten层
    x = Flatten()(x)
    
    # 全连接层
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    output = Dense(1)(x)
    
    catboost = CatBoostLayer()
    model = Model(inputs=model_input, outputs=output)

    return model, catboost


In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling1D, Add, Input
from keras.layers import LeakyReLU
from keras.models import Model

def residual_block(x, filters):
    shortcut = x
    x = Conv1D(filters, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv1D(filters, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x, shortcut])
    return x

def EcoCurrentNet_V4(input_shape, num_classes=5):
    model_inputs = Input(shape=(input_shape, 1))
    x = Conv1D(64, kernel_size=3, padding='same')(model_inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    # 添加残差块
    x = residual_block(x, 64)

    # 第二层卷积
    x = Conv1D(128, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    # 添加残差块
    x = residual_block(x, 128)

    # 第三层卷积
    x = Conv1D(256, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling1D(pool_size=2)(x)

    # 全局平均池化层
    x = GlobalAveragePooling1D()(x)
    
    # 全连接层
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # 输出层
    outputs = Dense(1)(x)

    model = Model(inputs=model_inputs, outputs=outputs)
    catboost = CatBoostLayer()
    
    return model, catboost


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling1D, Add, Input, Activation
from keras.layers import LeakyReLU, Attention
from keras.regularizers import l2

def residual_block(x, filters):
    shortcut = x
    x = Conv1D(filters, kernel_size=3, padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv1D(filters, kernel_size=3, padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Add()([x, shortcut])
    return x

def EcoCurrentNet_V5(input_shape):
    model_inputs = Input(shape=(input_shape, 1))

    # 第一层卷积
    x = Conv1D(64, kernel_size=3, padding='same', kernel_regularizer=l2(0.01))(model_inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    # 添加多个残差块
    for _ in range(2):  # 第一个残差块
        x = residual_block(x, 64)

    # 第二层卷积
    x = Conv1D(128, kernel_size=3, padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    # 添加多个残差块
    for _ in range(2):  # 第二个残差块
        x = residual_block(x, 128)

    # 第三层卷积
    x = Conv1D(256, kernel_size=3, padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling1D(pool_size=2)(x)

    # 添加多个残差块
    for _ in range(2):  # 第三个残差块
        x = residual_block(x, 256)

    # 全局平均池化层
    x = GlobalAveragePooling1D()(x)
    
    # 添加注意力机制
    attention = Attention()([x, x])
    x = Dense(512, activation='relu')(attention)
    x = Dropout(0.5)(x)

    # 全连接层
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # 输出层（回归任务）
    outputs = Dense(1)(x)

    model = Model(inputs=model_inputs, outputs=outputs)
    catboost = CatBoostLayer()
    return model, catboost

